In [6]:
import pandas as pd
import json

In [7]:
relative_path = "chessbuds_messages.json"

#with open(literal_path) as j:
 #   sp_dict = json.load(j)

with open('chessbuds_messages.json') as j:
    cb_dict = json.load(j)

Type will give you the type of the data

In [8]:
type(cb_dict)

dict

In [ ]:
cb_dict

In [8]:
cb_dict.keys() #keys available in the dictionary

dict_keys(['participants', 'messages', 'title', 'is_still_participant', 'thread_type', 'thread_path', 'magic_words', 'joinable_mode'])

In [9]:
cb_dict['participants']

[{'name': 'Scott Pence'},
 {'name': 'Chad Larson'},
 {'name': 'Joanna Rusch'},
 {'name': 'Angela Babbitt Pence'},
 {'name': 'David Silva'},
 {'name': 'Aaron Rusch'},
 {'name': 'Timothy Vanderpool'}]

In [10]:
cb_dict['messages'][0]

{'sender_name': 'Joanna Rusch',
 'timestamp_ms': 1666374933946,
 'content': "Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.",
 'reactions': [{'reaction': 'ð\x9f\x91\x8d', 'actor': 'Chad Larson'},
  {'reaction': 'ð\x9f\x91\x8d', 'actor': 'Chad Larson'}],
 'type': 'Generic',
 'is_unsent': False,
 'is_taken_down': False,
 'bumped_message_metadata': {'bumped_message': "Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.",
  'is_bumped': False}}

#### The chessbuds_messages json file seems to be a dictionary. Participants is list of dictionaries where each one has a key of name and value of a name. Messages is also a list of dictionaries. There are quite a few different key value pairs inside: timestamp_ms, content, reactions, type, is_unsent, is_taken_down, bumped_message_metadata and is_bumped. Inside some of those are further nested dictionaries. So we have the following:¶
* chess_buds_messages.json
    * participants (List)
        * name Name of the participant
    * messages (List)
        * sender_name Name of the message sender
        * timestamp_ms Timestamp of the message in milliseconds
        * content The text of the message
        * reactions Reactions to the message
            * reaction Actual reaction to the message
            * actor Name of the participant who reacted
        * type Type of the message (i.e. "Generic")
        * is_unsent Is the message unsent
        * is_taken_down Is the message was taken down
        * bumped_message_metadata Metadata about the bumped message
            * bumped_message Content of the bumped message
            * is_bumped Is the message bumped


#### I believe at this point that messages should be what is sent to the constructor as the participants are included in the sender_name. However, I would like to use sender_name as the index. This below code does not work because messages is a list, not a dictionary. I tried thinking about how I would use something like cb_dict['messages'][0] and somehow iterate over them. But I feel my next solution was better as it follows along with how me wrangled data in class.

In [11]:
cb_dict['messages']['sender_name']

TypeError: list indices must be integers or slices, not str

#### So I will take the contents of messages and store in a variable and then call the data frame constructor with that.

In [13]:
messages = cb_dict['messages']
df_chess = pd.DataFrame(messages)

In [14]:
pd.set_option('display.max_colwidth', None) #make the dataframe have no display column width so I can see full values

In [15]:
df_chess.head(20)

,sender_name,timestamp_ms,content,reactions,type,is_unsent,is_taken_down,bumped_message_metadata,share,photos,gifs,users
0,Joanna Rusch,1666374933946,"Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.","[{'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'ð', 'actor': 'Chad Larson'}]",Generic,False,False,"{'bumped_message': 'Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.', 'is_bumped': False}",NaN,NaN,NaN,NaN
1,Chad Larson,1666373448613,"To be fair to Hans....no one wants to be associated with an ""anal bead"" theory.","[{'reaction': 'ð', 'actor': 'Scott Pence'}, {'reaction': 'ð', 'actor': 'Scott Pence'}]",Generic,False,False,"{'bumped_message': 'To be fair to Hans....no one wants to be associated with an ""anal bead"" theory.', 'is_bumped': False}",NaN,NaN,NaN,NaN
2,Chad Larson,1666373216381,He would have to prove he didn't cheat and that they knew he didn't cheat....and made it all up. That's not provable.,"[{'reaction': 'ð', 'actor': 'Scott Pence'}, {'reaction': 'ð', 'actor': 'Scott Pence'}]",Generic,False,False,"{'bumped_message': 'He would have to prove he didn't cheat and that they knew he didn't cheat....and made it all up. That's not provable.', 'is_bumped': False}",NaN,NaN,NaN,NaN
3,Scott Pence,1666373164883,"Yeah, no way. You over shoot and hope to get a portion.","[{'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'ð', 'actor': 'Chad Larson'}]",Generic,False,False,"{'bumped_message': 'Yeah, no way. You over shoot and hope to get a portion.', 'is_bumped': False}",NaN,NaN,NaN,NaN
4,Chad Larson,1666373111157,"From what I see, I don't think he could win. But he is probably hoping for a good sized settlement",NaN,Generic,False,False,"{'bumped_message': 'From what I see, I don't think he could win. But he is probably hoping for a good sized settlement', 'is_bumped': False}",NaN,NaN,NaN,NaN
5,Scott Pence,1666372784166,https://youtu.be/EDvK6i86EZ0,"[{'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'ð', 'actor': 'Chad Larson'}]",Share,False,False,"{'bumped_message': 'https://youtu.be/EDvK6i86EZ0', 'is_bumped': False}",{'link': 'https://www.youtube.com/watch?v=EDvK6i86EZ0'},NaN,NaN,NaN
6,Chad Larson,1666319276656,Hans is suing Magnus and Hikaru for $100 million dollars hahaha,"[{'reaction': 'ð', 'actor': 'Joanna Rusch'}, {'reaction': 'ð', 'actor': 'Joanna Rusch'}]",Generic,False,False,"{'bumped_message': 'Hans is suing Magnus and Hikaru for $100 million dollars hahaha', 'is_bumped': False}",NaN,NaN,NaN,NaN
7,Scott Pence,1666197242777,My first great move!,NaN,Generic,False,False,"{'bumped_message': 'My first great move!', 'is_bumped': False}",NaN,NaN,NaN,NaN
8,Scott Pence,1666197230009,NaN,"[{'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'â¤', 'actor': 'Joanna Rusch'}, {'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'â¤', 'actor': 'Joanna Rusch'}]",Generic,False,False,"{'bumped_message': '', 'is_bumped': False}",NaN,"[{'uri': 'messages/inbox/chessbuds_npjakt9u1g/photos/312097852_642845703998797_5890449692529901285_n_642845700665464.png', 'creation_timestamp': 1666197228}]",NaN,NaN
9,Scott Pence,1666052367051,NaN,"[{'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'ð®', 'actor': 'Joanna Rusch'}, {'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'ð®', 'actor': 'Joanna Rusch'}]",Generic,False,False,"{'bumped_message': '', 'is_bumped': False}",NaN,"[{'uri': 'messages/inbox/chessbuds_npjakt9u1g/photos/311428108_3370460183189495_4558735131754408614_n_3370460179856162.png', 'creation_timestamp': 1666052365}]",NaN,NaN


##### I think in this case the observable unit is the message and then we have observations as the columns. However, what to do with the columns that are still lists. I don't want to include reactions at this point because I find them unhelpful and I am not sure how to work with them yet. A potential use for reaction data in the future may be tracking overall sentiment for a message(this many likes, angry faces, etc). Now, I will work on the metadata column.

In [16]:
df_chess['bumped_message_metadata'].iloc[0]

{'bumped_message': "Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.",
 'is_bumped': False}

##### Message is just the original message repeated. Going to try to just use the boolean portion using str.get to access the value in the key value pair. I am doing it this way because the value is a string and not an actual boolean value. and display if a message is_bumped. This could also potentially be used to display message sentiment or engagement.

In [17]:
df_chess['is_bumped'] = df_chess['bumped_message_metadata'].str.get('is_bumped')

In [18]:
df_chess.head()

,sender_name,timestamp_ms,content,reactions,type,is_unsent,is_taken_down,bumped_message_metadata,share,photos,gifs,users,is_bumped
0,Joanna Rusch,1666374933946,"Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.","[{'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'ð', 'actor': 'Chad Larson'}]",Generic,False,False,"{'bumped_message': 'Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.', 'is_bumped': False}",NaN,NaN,NaN,NaN,False
1,Chad Larson,1666373448613,"To be fair to Hans....no one wants to be associated with an ""anal bead"" theory.","[{'reaction': 'ð', 'actor': 'Scott Pence'}, {'reaction': 'ð', 'actor': 'Scott Pence'}]",Generic,False,False,"{'bumped_message': 'To be fair to Hans....no one wants to be associated with an ""anal bead"" theory.', 'is_bumped': False}",NaN,NaN,NaN,NaN,False
2,Chad Larson,1666373216381,He would have to prove he didn't cheat and that they knew he didn't cheat....and made it all up. That's not provable.,"[{'reaction': 'ð', 'actor': 'Scott Pence'}, {'reaction': 'ð', 'actor': 'Scott Pence'}]",Generic,False,False,"{'bumped_message': 'He would have to prove he didn't cheat and that they knew he didn't cheat....and made it all up. That's not provable.', 'is_bumped': False}",NaN,NaN,NaN,NaN,False
3,Scott Pence,1666373164883,"Yeah, no way. You over shoot and hope to get a portion.","[{'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'ð', 'actor': 'Chad Larson'}]",Generic,False,False,"{'bumped_message': 'Yeah, no way. You over shoot and hope to get a portion.', 'is_bumped': False}",NaN,NaN,NaN,NaN,False
4,Chad Larson,1666373111157,"From what I see, I don't think he could win. But he is probably hoping for a good sized settlement",NaN,Generic,False,False,"{'bumped_message': 'From what I see, I don't think he could win. But he is probably hoping for a good sized settlement', 'is_bumped': False}",NaN,NaN,NaN,NaN,False


#### It is at this point that I remembered list comprehension. Similar to our example in class, however, I am using get instead of join. Get will get the value in the key value pair of is_bumped. This list comprehension terates through each row in the bumped_message_metadata column, if one exists it then will try to get the value for the key is_bumped (if true true, if false false, if there is no data then it will return an empty string) if there is no item in x the else will trigger and also return an empty string.

In [19]:
df_chess['is_bumped'] = [x.get('is_bumped', '') if x else '' for x in df_chess['bumped_message_metadata']]

In [20]:
print(df_chess['bumped_message_metadata'].head(10))

0    {'bumped_message': 'Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.', 'is_bumped': False}
1                                                                                                    {'bumped_message': 'To be fair to Hans....no one wants to be associated with an "anal bead" theory.', 'is_bumped': False}
2                                                             {'bumped_message': 'He would have to prove he didn't cheat and that they knew he didn't cheat....and made it all up.  That's not provable.', 'is_bumped': False}
3                                                                                                                            {'bumped_message': 'Yeah, no way. You over shoot and hope to get a portion.', 'is_bumped': False}
4                                                                                {'bumped_message': 'From wh

##### It seems all of the messages have an is_bumped value of false but the messages indicate that they were bumped? Not sure. For now I will just display is_bumped value as original

In [21]:
df_chess.head()

,sender_name,timestamp_ms,content,reactions,type,is_unsent,is_taken_down,bumped_message_metadata,share,photos,gifs,users,is_bumped
0,Joanna Rusch,1666374933946,"Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.","[{'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'ð', 'actor': 'Chad Larson'}]",Generic,False,False,"{'bumped_message': 'Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.', 'is_bumped': False}",NaN,NaN,NaN,NaN,False
1,Chad Larson,1666373448613,"To be fair to Hans....no one wants to be associated with an ""anal bead"" theory.","[{'reaction': 'ð', 'actor': 'Scott Pence'}, {'reaction': 'ð', 'actor': 'Scott Pence'}]",Generic,False,False,"{'bumped_message': 'To be fair to Hans....no one wants to be associated with an ""anal bead"" theory.', 'is_bumped': False}",NaN,NaN,NaN,NaN,False
2,Chad Larson,1666373216381,He would have to prove he didn't cheat and that they knew he didn't cheat....and made it all up. That's not provable.,"[{'reaction': 'ð', 'actor': 'Scott Pence'}, {'reaction': 'ð', 'actor': 'Scott Pence'}]",Generic,False,False,"{'bumped_message': 'He would have to prove he didn't cheat and that they knew he didn't cheat....and made it all up. That's not provable.', 'is_bumped': False}",NaN,NaN,NaN,NaN,False
3,Scott Pence,1666373164883,"Yeah, no way. You over shoot and hope to get a portion.","[{'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'ð', 'actor': 'Chad Larson'}]",Generic,False,False,"{'bumped_message': 'Yeah, no way. You over shoot and hope to get a portion.', 'is_bumped': False}",NaN,NaN,NaN,NaN,False
4,Chad Larson,1666373111157,"From what I see, I don't think he could win. But he is probably hoping for a good sized settlement",NaN,Generic,False,False,"{'bumped_message': 'From what I see, I don't think he could win. But he is probably hoping for a good sized settlement', 'is_bumped': False}",NaN,NaN,NaN,NaN,False


##### Drop reactions, bumped_message_metadata and gifs.

In [22]:
df_chess = df_chess.drop(columns=['reactions', 'bumped_message_metadata', 'gifs', 'users'])

##### Just realized that users has NaN as well as lists of dictionaries. Using logic similar to what we discussed at the end of class however we could in fact break out the user names into their own column. I have just ran out of time to completely work through that logic. I tried a few things but it always broke at any NaN value because we were assuming all of the values would be a list of dictionaries.

In [23]:
df_chess

,sender_name,timestamp_ms,content,type,is_unsent,is_taken_down,share,photos,is_bumped
0,Joanna Rusch,1666374933946,"Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.",Generic,False,False,NaN,NaN,False
1,Chad Larson,1666373448613,"To be fair to Hans....no one wants to be associated with an ""anal bead"" theory.",Generic,False,False,NaN,NaN,False
2,Chad Larson,1666373216381,He would have to prove he didn't cheat and that they knew he didn't cheat....and made it all up. That's not provable.,Generic,False,False,NaN,NaN,False
3,Scott Pence,1666373164883,"Yeah, no way. You over shoot and hope to get a portion.",Generic,False,False,NaN,NaN,False
4,Chad Larson,1666373111157,"From what I see, I don't think he could win. But he is probably hoping for a good sized settlement",Generic,False,False,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...
218,Scott Pence,1615855643693,"Thank you, thank you.",Generic,False,False,NaN,NaN,False
219,Scott Pence,1615855620614,NaN,Generic,False,False,NaN,NaN,False
220,Chad Larson,1615854725212,Chad Larson added Angela Babbitt Pence to the group.,Subscribe,False,False,NaN,NaN,False
221,Chad Larson,1615854680797,"Scott studied my games, did a specific opening for me, and ran me over! It was painful haha",Generic,False,False,NaN,NaN,False


##### I think our data is now in tidy format. With each row being a observable unit of a message and the columns being variables.

### Out of curiosity I decided to try a library version of flattening data using json_normalize. It did a somewhat decent job of flattening the data. However, as is visible there are some issues such as: there still is a column with multiple entries in the same row, headings of columns are hard to understand(which would possibly make them hard to analyze), NaN was added to observations that were missing(which could be convenient if that is what you need) which may not be what our intended choice is regarding missingness (amongst other issues).
### Links used to determine how to flatten JSON data. Additionally, I try to learn the basics before using a tool like normalize. Otherwise, you may have no idea how a tool works. It is like learning data strucutres by creating your own first manually then appreciating just being able to use a prebuilt string or int or list for example.
#### __[Medium Article on Flattening JSON](https://medium.com/@avishek2020/exploring-the-power-of-json-normalize-a-step-by-step-guide-to-flattening-complex-json-data-in-5a2d694dd26)__
#### __[Normalize Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html#pandas.json_normalize)__

In [24]:
from pandas import json_normalize

In [25]:
df_chessbuds_flat = json_normalize(cb_dict['messages'], sep='_')
df_chessbuds_flat.head()

,sender_name,timestamp_ms,content,reactions,type,is_unsent,is_taken_down,bumped_message_metadata_bumped_message,bumped_message_metadata_is_bumped,share_link,photos,gifs,users
0,Joanna Rusch,1666374933946,"Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.","[{'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'ð', 'actor': 'Chad Larson'}]",Generic,False,False,"Maybe he just wants to ride the publicity for a bit longer, even if he doesn't get any money from the lawsuit. Like, I didn't know his name before this but I certainly do now.",False,NaN,NaN,NaN,NaN
1,Chad Larson,1666373448613,"To be fair to Hans....no one wants to be associated with an ""anal bead"" theory.","[{'reaction': 'ð', 'actor': 'Scott Pence'}, {'reaction': 'ð', 'actor': 'Scott Pence'}]",Generic,False,False,"To be fair to Hans....no one wants to be associated with an ""anal bead"" theory.",False,NaN,NaN,NaN,NaN
2,Chad Larson,1666373216381,He would have to prove he didn't cheat and that they knew he didn't cheat....and made it all up. That's not provable.,"[{'reaction': 'ð', 'actor': 'Scott Pence'}, {'reaction': 'ð', 'actor': 'Scott Pence'}]",Generic,False,False,He would have to prove he didn't cheat and that they knew he didn't cheat....and made it all up. That's not provable.,False,NaN,NaN,NaN,NaN
3,Scott Pence,1666373164883,"Yeah, no way. You over shoot and hope to get a portion.","[{'reaction': 'ð', 'actor': 'Chad Larson'}, {'reaction': 'ð', 'actor': 'Chad Larson'}]",Generic,False,False,"Yeah, no way. You over shoot and hope to get a portion.",False,NaN,NaN,NaN,NaN
4,Chad Larson,1666373111157,"From what I see, I don't think he could win. But he is probably hoping for a good sized settlement",NaN,Generic,False,False,"From what I see, I don't think he could win. But he is probably hoping for a good sized settlement",False,NaN,NaN,NaN,NaN


1- In a Markdown cell, describe what you have set the row-level (observation) to represent and what you have set each column to represent
The row represents an individual message and the columns represent observations on those messages.

2- Explain why your final DataFrame meets tidy data principles
The final data frame meets tiny data principles because each row represents a single observable(message) and the columns represent variables associated with those messages broken out into seperate columns and this observational unit of let's call it a conversation is now in its own table.

3- What are (if any) alternative, tidy formats that would also be acceptable for this data?
If we knew the reactions(what they stand for) we pull those back in and then break them out into individual columns inside our data frame. I discussed this earlier in the notebook, as this could be used to track sentiment and engagment possibly. This also has me thinking what if we used participants in the call to the data frame constructor. This could allow us to track participant engagement levels. This could also allow us to track things like what their typical reactions are, what time do they usually respond, if they have many is_taken_down maybe that could tell us that they are a "toxic" user if it's the case that taken down means an admin deleted it, or if it's the case that it means the participant removed it themselves, maybe that tells us that they are indecisive. Or potentially show that certain users have high engagement with their messages, in which case we could (if we were a business) offer those participants some type of upgraded membership.

In a Markdown cell, describe a potential visualization that would let someone explore and, ultimately, better understand this data
Be specific citing the variables available in the data and how the information would be mapped to the visual elements.
Theoretically, we could create a dashboard with multiple different visualizations. However, a single visualization for this single data frame, could be a heat map. Something I can imagine wanting to understand is the timeing of users engagement. Therefore we could make the x-axis time of day (we woulad have to convert Unix ms to regular time which I would imagine is possible), then the y-axis can be participant names, the number of messages would be represented by a color(and also a number for further clarity). Now we can see where the people involved with this message are most or least active at certain times of days.